In [1]:
import matplotlib.pyplot as plt
from models import CnnModel, CrnnLongModel, CrnnModel, RnnModel, MambaVisionModel
from train import main_train, validate_test, record_matrix
from Paras import Para
from data_loader import torch_dataset_loader
import torch

# MambaVision model
## Define Paras

In [2]:

Para.learning_rate = 1e-5
Para.batch_size = 20
Para.epoch_num = 50

train_loader = torch_dataset_loader(Para.TRAIN_DATA_PATH, Para.batch_size, True, Para.kwargs)
validation_loader = torch_dataset_loader(Para.VAL_DATA_PATH, Para.batch_size, False, Para.kwargs)
test_loader = torch_dataset_loader(Para.TEST_DATA_PATH, Para.batch_size, False, Para.kwargs)

In [3]:
MambaVision = MambaVisionModel()

## Train Model

In [4]:
res = main_train(model=MambaVision, 
                 train_loader=train_loader,
                 valid_loader=validation_loader,
                 log_name='MambaVisionModel.json',
                 save_name='MambaVisionModel.pt',
                 lr=Para.learning_rate,
                 epoch_num=Para.epoch_num)

inp out: torch.Size([20, 1, 448, 128])
chunk out: torch.Size([20, 1, 90, 128])
fetures out: torch.Size([20, 640, 3, 4])
mamba in: torch.Size([20, 6, 7680])


ValueError: only one element tensors can be converted to Python scalars

In [ ]:
plt.figure()
plt.plot(res['train_accu'], c='b', label='Training set accuracy')
plt.plot(res['valid_accu'], c='r', label='Validation set accuracy')
plt.title('Accuracy vs Epochs / MambaVision Model')
plt.legend()
plt.show()

## Test Model

In [ ]:
Para.MODEL_SAVE_FOlD = './'
MambaVision.cuda().load_state_dict(torch.load(Para.MODEL_SAVE_FOlD + 'modelsMambaVisionModel.pt'))
validate_test(model=MambaVision, epoch=0, use_loader=test_loader)

In [ ]:
MambaVision.cuda().load_state_dict(torch.load(Para.MODEL_SAVE_FOlD + 'modelsMambaVisionModel.pt'))
record_matrix(model=MambaVision, use_loader=test_loader, log_name='MambaVisionMatrix.json')

# CNN model
## Define Paras

In [ ]:
Para.learning_rate = 1e-5
Para.batch_size = 20
Para.epoch_num = 50

train_loader = torch_dataset_loader(Para.TRAIN_DATA_PATH, Para.batch_size, True, Para.kwargs)
validation_loader = torch_dataset_loader(Para.VAL_DATA_PATH, Para.batch_size, False, Para.kwargs)
test_loader = torch_dataset_loader(Para.TEST_DATA_PATH, Para.batch_size, False, Para.kwargs)

In [ ]:
CNN = CnnModel()

## Train Model

In [ ]:
res = main_train(model=CNN, 
                 train_loader=train_loader,
                 valid_loader=validation_loader,
                 log_name='CnnModel.json',
                 save_name='CnnModel.pt',
                 lr=Para.learning_rate,
                 epoch_num=Para.epoch_num)

In [ ]:
plt.figure()
plt.plot(res['train_accu'], c='b', label='Training set accuracy')
plt.plot(res['valid_accu'], c='r', label='Validation set accuracy')
plt.title('Accuracy vs Epochs / CNN Model')
plt.legend()
plt.show()

## Test Model

In [ ]:
CNN.cuda().load_state_dict(torch.load(Para.MODEL_SAVE_FOlD + 'CnnModel.pt'))
validate_test(model=CNN, epoch=0, use_loader=test_loader)

In [ ]:
CNN.cuda().load_state_dict(torch.load(Para.MODEL_SAVE_FOlD + 'CnnModel.pt'))
record_matrix(model=CNN, use_loader=test_loader, log_name='CnnMatrix.json')

# CRNN Model
## Paras

In [ ]:
Para.learning_rate = 1e-5
Para.batch_size = 20
Para.epoch_num = 60

In [ ]:
CRNN = CrnnModel()

## Train

In [ ]:
res = main_train(model=CRNN, 
                 train_loader=train_loader,
                 valid_loader=validation_loader,
                 log_name='CrnnModel.json',
                 save_name='CrnnModel.pt',
                 lr=Para.learning_rate,
                 epoch_num=Para.epoch_num)

## Test Model

In [ ]:
plt.figure()
plt.plot(res['train_accu'], c='b', label='Training set accuracy')
plt.plot(res['valid_accu'], c='r', label='Validation set accuracy')
plt.title('Accuracy vs Epochs / CRNN Model')
plt.legend()
plt.show()

In [ ]:
CRNN.cuda().load_state_dict(torch.load(Para.MODEL_SAVE_FOlD + 'CrnnModel.pt'))
validate_test(model=CRNN, epoch=0, use_loader=test_loader)

In [ ]:
CRNN.cuda().load_state_dict(torch.load(Para.MODEL_SAVE_FOlD + 'CrnnModel.pt'))
record_matrix(model=CRNN, use_loader=test_loader, log_name='CrnnMatrix.json')

# LCRNN Model
## Paras

In [ ]:
Para.learning_rate = 1e-5
Para.batch_size = 20
Para.epoch_num = 60

train_loader = torch_dataset_loader(Para.LA_TRAIN_DATA_PATH, Para.batch_size, True, Para.kwargs)
validation_loader = torch_dataset_loader(Para.LA_VAL_DATA_PATH, Para.batch_size, False, Para.kwargs)
test_loader = torch_dataset_loader(Para.LA_TEST_DATA_PATH, Para.batch_size, False, Para.kwargs)

LCRNN = CrnnLongModel()

## Train

In [ ]:
res = main_train(model=LCRNN, 
                 train_loader=train_loader,
                 valid_loader=validation_loader,
                 log_name='CrnnLongModel.json',
                 save_name='CrnnLongModel.pt',
                 lr=Para.learning_rate,
                 epoch_num=Para.epoch_num)

## Test Model

In [ ]:
plt.figure()
plt.plot(res['train_accu'], c='b', label='Training set accuracy')
plt.plot(res['valid_accu'], c='r', label='Validation set accuracy')
plt.title('Accuracy vs Epochs / CRNN Model')
plt.legend()
plt.show()

In [ ]:
LCRNN.cuda().load_state_dict(torch.load(Para.MODEL_SAVE_FOlD + 'CrnnLongModel.pt'))
validate_test(model=LCRNN, epoch=0, use_loader=test_loader)

In [ ]:
LCRNN.cuda().load_state_dict(torch.load(Para.MODEL_SAVE_FOlD + 'CrnnLongModel.pt'))
record_matrix(model=LCRNN, use_loader=test_loader, log_name='CrnnLongMatrix.json')

# RNN Model

In [ ]:
Para.learning_rate = 1e-5
Para.batch_size = 32
Para.epoch_num = 40

train_loader = torch_dataset_loader(Para.LA_TRAIN_DATA_PATH, Para.batch_size, True, Para.kwargs)
validation_loader = torch_dataset_loader(Para.LA_VAL_DATA_PATH, Para.batch_size, False, Para.kwargs)
test_loader = torch_dataset_loader(Para.LA_TEST_DATA_PATH, Para.batch_size, False, Para.kwargs)

RNN = RnnModel()

In [ ]:
res = main_train(model=RNN, 
                 train_loader=train_loader,
                 valid_loader=validation_loader,
                 log_name='RnnModel.json',
                 save_name='RnnModel.pt',
                 lr=Para.learning_rate,
                 epoch_num=Para.epoch_num)

In [ ]:
plt.figure()
plt.plot(res['train_accu'], c='b', label='Training set accuracy')
plt.plot(res['valid_accu'], c='r', label='Validation set accuracy')
plt.title('Accuracy vs Epochs / RCNN Model')
plt.legend()
plt.show()

In [ ]:
RNN.cuda().load_state_dict(torch.load(Para.MODEL_SAVE_FOlD + 'RnnModel.pt'))
validate_test(model=RNN, epoch=0, use_loader=test_loader)